In [ ]:
# 1. Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Cargar dataset
url = “https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv”
df = pd.read_csv(url, parse_dates=[‘Date’])

# 3. Información general
df.info()
df.describe()

# 4. Valores nulos
df.isnull().sum()

# 5. Imputar o eliminar si es necesario
df = df.dropna()

# 6. Visualización general de la serie
df['Temp'].plot(figsize=(14,5), title="Temperatura diaria en Melbourne")
plt.ylabel("Temperatura (°C)")
plt.show()

# 7. Histogramas y boxplots
plt.figure(figsize=(12,4))
sns.histplot(df['Temp'], bins=50, kde=True)
plt.title("Distribución de temperaturas")
plt.show()

plt.figure(figsize=(10,3))
sns.boxplot(x=df['Temp'])
plt.title("Boxplot de temperaturas")
plt.show()

# 8. Correlación (si hay más variables)
if df.shape[1] > 1:
    sns.heatmap(df.corr(), annot=True)


In [ ]:
# 1. Librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


# 2. Normalización
scaler = MinMaxScaler()
df['Temp_scaled'] = scaler.fit_transform(df[['Temp']])

# 3. Transformar en secuencias
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_len = 30
X, y = create_sequences(df['Temp_scaled'].values, seq_len)

# 4. División train/test (80/20 temporal)
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# 5. Guardar datos
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)


In [ ]:
# 1. Librerías
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import mlflow
import mlflow.tensorflow

# 2. Cargar datos
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# 3. Construcción del modelo
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),
    Dense(1)
])
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 4. Callback MLflow
mlflow.tensorflow.autolog()

# 5. Entrenamiento
with mlflow.start_run():
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)


In [ ]:
# 1. Librerías
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import mlflow
import mlflow.tensorflow

# 2. Cargar datos
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# 3. Cargar modelo (opcional si ya está entrenado)
# model = load_model("lstm_model.h5")

# 4. Predicción
y_pred = model.predict(X_test)

# 5. Métricas
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("MSE:", mse)
print("MAE:", mae)

# 6. Exportar modelo y scaler
model.save("lstm_model.h5")
joblib.dump(scaler, "scaler.pkl")

# 7. Logging MLflow
with mlflow.start_run():
    mlflow.log_param("model_type", "LSTM")
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.tensorflow.log_model(model, artifact_path="model")
    mlflow.log_artifact("scaler.pkl")

print("Modelo y artefactos registrados con éxito en MLflow")


